In [29]:
import numpy as np
import cv2 as cv
from utility import parse_xml
import pandas as pd
import glob
import os
from tqdm.auto import tqdm
import shutil
PATH_IMAGES = r"C:\project\Computer_Vision\term_paper_2023\datasets\antiuav_train\img"
PATH_LABELS = r"C:\project\Computer_Vision\term_paper_2023\datasets\antiuav_train\xml"
PATH_SAVE =  r"C:\project\Computer_Vision\term_paper_2023\datasets\background_from_antiUAV"
# img = cv.imread('messi_2.jpg')
# mask = cv.imread('mask2.png', cv.IMREAD_GRAYSCALE)
# dst = cv.inpaint(img,mask,3,cv.INPAINT_TELEA)
# cv.imshow('dst',dst)
# cv.waitKey(0)
# cv.destroyAllWindows()
if os.path.exists(PATH_SAVE):
    shutil.rmtree(PATH_SAVE)
os.makedirs(PATH_SAVE)

In [30]:
def get_df(images_path, labels_path):
    df = pd.DataFrame (glob.glob( os.path.join(images_path,'*.jpg')), columns=['path_image'])
    df["id"] = df['path_image'].apply(lambda x : x.split('\\')[-1].split('.')[0])
    df["path_label"] = df['id'].apply(lambda x : os.path.join(labels_path,f"{x}.xml"))
    return df

In [31]:
df = get_df(PATH_IMAGES,PATH_LABELS)

In [32]:

# mask = np.zeros(image.shape[:2], dtype="uint8")
# cv2.rectangle(mask, (0, 90), (290, 450), 255, -1)
# cv2.imshow("Rectangular Mask", mask)
# # apply our mask -- notice how only the person in the image is
# # cropped out
# masked = cv2.bitwise_and(image, image, mask=mask)
# cv2.imshow("Mask Applied to Image", masked)
# cv2.waitKey(0)

In [33]:
for i, row in tqdm(df.iterrows()):
        coords_drons , width_image, height_image = parse_xml(row['path_label'])
        image = cv.imread(row['path_image'])
        mask = np.zeros(image.shape[:2], dtype="uint8")
        flag = True
        for xmin, ymin, xmax, ymax in coords_drons:
            area = (xmax-xmin)*(ymax-ymin)/width_image/height_image
            if  area > 0.0011:
                flag = False
                break
            cv.rectangle(mask, (xmin-5, ymin-5), (xmax+5, ymax+5), 255, -1)
        if flag:
            image = cv.inpaint(image,mask,3,cv.INPAINT_TELEA)
            cv.imwrite(os.path.join(PATH_SAVE,f'{row["id"]}.jpg'), image)

0it [00:00, ?it/s]